# Use camera and install lib

In [ ]:
##read image
import cv2
image_file='xxx.jpg'
image = cv2.imread(image_file)
##show image
cv2.imshow("window name",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
##get real-time image from camera
# Define a video capture object
#cv2.VideoCapture(0):own computer's webcamera
cap = cv2.VideoCapture(1) #Resolution (640,480):camera provided in class
while True:
    ret, frame = cap.read() #returns ret and the frame
    cv2.imshow("window name",frame)
    
    #press q to end the while loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
          break
            
cv2.destroyAllWindows()

In [ ]:
##install library for aruco marker
pip install opencv-contrib-python --user

# Construct global map

In [ ]:
def compute_globalmap(image,start_id=0,goal_id=1):
    #Generate globalmap 0:free 1:occupied 2:start 3:goal
    #height, width = image.shape[:2]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #Simple threshold method
    #ret,thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

    # Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(gray,(5,5),0)
    ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    #Generate Global map
    #from 0,255 convert to 0(free),1(occupied);change to (x,y) coordinate
    globalmap=np.transpose(np.array(thresh<255,dtype=int))
    
    #Detect start and goal by aruco marker
    #Return start and goal position in (x,y) coordinate; if not detected return None
    #start_id: start aruco marker id
    #goal_id=: goal aruco marker id

    #Define aruco marker detector
    arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    arucoParams = cv2.aruco.DetectorParameters_create()

    #Detect aruco marker
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
    # print(corners, ids, rejected)
    start=None
    goal=None
    if ids is not None:
        for corner,number in zip(corners,ids):
            (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
            #print(topLeft, topRight, bottomRight, bottomLeft)
            centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
            if number==start_id:
                start=centerpoint
                #print("start:",start)
                globalmap[bottomRight[0]:topLeft[0]+1,topLeft[1]:bottomRight[1]+1]=2
                #print(globlemap[bottomRight[0]:topLeft[0],topLeft[1]:bottomRight[1]])

            elif number==goal_id:
                goal=centerpoint
                #print("goal:",goal)
                globalmap[bottomRight[0]:topLeft[0]+1,topLeft[1]:bottomRight[1]+1]=3
                #print(globlemap[bottomRight[0]:topLeft[0],topLeft[1]:bottomRight[1]])

            else:
                pass
    else: pass
    if start is None: print("NOT detect start point") 
    else:pass
    if goal is None: print("NOT detect start point") 
    else:pass
    
    
    return globalmap

def get_start(image, start_id=0):
    #Detect start by aruco marker
    #Return start position in (x,y) coordinate; if not detected return None
    #start_id: start aruco marker id

    #Define aruco marker detector
    arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    arucoParams = cv2.aruco.DetectorParameters_create()

    #Detect aruco marker
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
    # print(corners, ids, rejected)
    start=None
    if ids is not None:
        for corner,number in zip(corners,ids):
            (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
            #print(topLeft, topRight, bottomRight, bottomLeft)
            centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
            if number==start_id:
                start=centerpoint
                break
            else:
                pass
    else: pass
    if start is None: print("NOT detect start point") 
    else:pass
    
    return start

def get_goal(image, goal_id=1):
    #Detect goal by aruco marker
    #Return goal position in (x,y) coordinate; if not detected return None
    #goal_id=: goal aruco marker id

    #Define aruco marker detector
    arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    arucoParams = cv2.aruco.DetectorParameters_create()

    #Detect aruco marker
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
    # print(corners, ids, rejected)
    goal=None
    if ids is not None:
        for corner,number in zip(corners,ids):
            (topLeft, topRight, bottomRight, bottomLeft) = corner[0].astype(int)
            #print(topLeft, topRight, bottomRight, bottomLeft)
            centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
            if number==goal_id:
                goal=centerpoint
                break
            else:
                pass
    else: pass
    if goal is None: print("NOT detect start point") 
    else:pass
    
    return goal
    

# Get robot position and angle

In [ ]:
def get_robot_position(image,robot_id=3):
    #Detect robot position by aruco marker
    #Return robot position; if not detected return None
    #robot_id: robot aruco marker id

    #Define aruco marker detector
    arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    arucoParams = cv2.aruco.DetectorParameters_create()

    robot=None

    #Detect Aruco marker
    (corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict,parameters=arucoParams)
    # print(corners, ids, rejected)
    if ids is not None:
        #print("aruco marker detected:",ids)
        for corner,number in zip(corners,ids):
            (topLeft, topRight, bottomRight, bottomLeft) = corner[0]
            centerpoint=(int((topLeft[0]+bottomRight[0])/2),int((topLeft[1]+bottomRight[1])/2))
            if number==robot_id:
                robot_position=centerpoint
                angle=angle_calculate(topLeft,topRight)
                break
            else:pass
    else:pass
    
    return robot_position,angle

def angle_calculate(pt1,pt2):  # function which returns angle between two points in the range of 0-359
    angle_list = list(range(0,360,1))
    x=pt2[0]-pt1[0] 
    y=pt2[1]-pt1[1]
    angle=int(math.degrees(math.atan2(y,x))) #takes 2 points nad give angle with respect to horizontal axis in range(-180,180)
    angle = angle_list[angle]
    
    return int(angle)